## RAKE
С помощью алгоритма RAKE можно найти наиболее вероятные ключевые слова и словосочетания в каждом отдельном тексте, безотносительно ко всей коллекции документов. 
Идея состоит в том, чтобы найти самые частотные кандитаты в ключевые словосочетания (2 или больше слов) по всей коллекции текстов копуса твитов для sentiment-анализа. Мы предполагаем, что наиболее частотные ключевые словосочетания скорее всего будут оценочными из-за специфики корпуса (эмоционально окрашенные тексты с положительной или отрицательной поляности). 
В итоге было решено оставить в итоговом списке только те словосочетания, которые входят в 1000 наиболее частотных и содержат хотя бы одно слово из уже собранного нами списка однословной оценочной лексики. 

In [1]:
import RAKE

In [3]:
import json

In [7]:
import numpy as np
import pandas as pd

In [18]:
import regex as re

In [22]:
from preprocessing import tokenize, remove_nickname_, lemmatize_pos_tag

In [41]:
from collections import Counter

In [4]:
with open('stop_words.json') as file:
        stop_words = json.load(file)

In [56]:
good_seed_tokens = [
 ['вежливый_ADJ'],
 ['великолепный_ADJ'],
 ['вкусный_ADJ'],
 ['внимательный_ADJ'],
 ['вау_NOUN'],
 ['неплохой_ADJ'],
 ['идеальный_ADJ'],
 ['неприятный_ADJ'],
 ['понравиться_VERB'],
 ['рекомендовать_VERB'],
 ['ура_PART'],
 ['советовать_VERB'],
 ['отвратительный_ADJ'],
 ['отличный_ADJ'],
 ['положительный_ADJ'],
 ['отрицательный_ADJ'],
 ['приятный_ADJ'],
 ['спасибо_NOUN'],
 ['радовать_VERB'],
 ['молодец_NOUN'],
 ['улыбаться_VERB'],
 ['приветливый_ADJ'],
 ['отличный_ADJ'],
 ['замечательный_ADJ']]

In [11]:
neg = pd.read_csv('negative.csv', sep = ';', header=None)
pos =  pd.read_csv('positive.csv', sep = ';', header=None)

In [16]:
strip_characters = '`«»‰~@\'▲▼„"#$%^&*()_+=[]\{\}°|•<>'

In [25]:
# лемматизируем сами слова, но оставляем знаки препинания
def lemmatize_only(text):
    text = text.lower()
    text = remove_nickname_(text)
    words = re.findall('[а-яёА-ЯЁ]+', text)
    words = set(words)
    for word in words:
        text = text.replace(word, lemmatize_pos_tag(word, need_pos=False))
    return text

In [27]:
texts = []
for corp in [neg, pos]:
    for row in corp[3]:
        texts.append(lemmatize_only(row))

In [29]:
Rake = RAKE.Rake(stop_words)

In [31]:
res = Rake.run(texts[10], maxWords = 4)

In [34]:
res

[('хоть знать', 4.0),
 ('ебать', 1.0),
 ('фотка', 1.0),
 ('про', 1.0),
 ('молчать', 1.0),
 ('вообще', 1.0),
 ('тп', 1.0),
 ('выглядеть', 1.0)]

In [40]:
results = []
for text in texts:
    res = Rake.run(text, maxWords = 4)
    results.extend([i[0] for i in res[:3]])

In [48]:
results_clean = []
for i in results:
    if re.findall('[а-я]+', i) != []:
        results_clean.append(i)

In [49]:
cnt = Counter(results_clean)

In [50]:
results_multiword = [i for i in results_clean if ' ' in i]

In [51]:
cnt_multiword = Counter(results_multiword)

In [52]:
cnt.most_common(100)

[('быть', 3471),
 ('знать', 2704),
 ('хотеть', 1987),
 ('блин', 1862),
 ('еще', 1853),
 ('нет', 1715),
 ('спасибо', 1464),
 ('понимать', 1397),
 ('если', 1394),
 ('думать', 1332),
 ('любить', 1318),
 ('сегодня', 1251),
 ('хорошо', 1142),
 ('тоже', 1053),
 ('мочь', 1018),
 ('ахах', 990),
 ('вообще', 935),
 ('говорить', 920),
 ('уже', 893),
 ('сказать', 875),
 ('новый год', 833),
 ('делать', 810),
 ('завтра', 796),
 ('школа', 757),
 ('день', 746),
 ('добрый утро', 738),
 ('сидеть', 730),
 ('смотреть', 714),
 ('день рождение', 712),
 ('человек', 708),
 ('надо', 663),
 ('бля', 659),
 ('ахи', 655),
 ('жаль', 592),
 ('писать', 587),
 ('кажется', 579),
 ('видеть', 577),
 ('пойти', 563),
 ('плохо', 558),
 ('спать', 556),
 ('скучать', 547),
 ('привет', 540),
 ('приходить', 538),
 ('дело', 538),
 ('надеяться', 527),
 ('давать', 524),
 ('нравиться', 522),
 ('идти', 498),
 ('жизнь', 493),
 ('больше', 485),
 ('друг', 483),
 ('ага', 475),
 ('сделать', 456),
 ('написать', 455),
 ('хороший', 444),
 ('

In [53]:
cnt_multiword.most_common(100)

[('новый год', 833),
 ('добрый утро', 738),
 ('день рождение', 712),
 ('быть хорошо', 257),
 ('спокойный ночь', 257),
 ('#євромайдан http', 215),
 ('хотеть спать', 205),
 ('может быть', 202),
 ('последний время', 190),
 ('первый раз', 177),
 ('надо быть', 169),
 ('тоже хотеть', 158),
 ('идиот целый день', 156),
 ('офигенныйй день', 156),
 ('день позитив', 156),
 ('новогодний настроение', 155),
 ('быть скучать', 147),
 ('ложиться спать', 145),
 ('хороший настроение', 143),
 ('приходить домой', 138),
 ('хороший день', 137),
 ('если честно', 136),
 ('очень плохо', 130),
 ('голова болеть', 129),
 ('очень жаль', 128),
 ('сладкий сон', 124),
 ('спасибо большой', 123),
 ('нужный ли', 112),
 ('новое год', 109),
 ('скоро новый год', 106),
 ('ненавидеть пробка ретвлять', 105),
 ('быть весело', 99),
 ('дед мороз', 98),
 ('сидеть дома', 95),
 ('можно быть', 93),
 ('мочь уснуть', 90),
 ('жизнь боль', 90),
 ('очень любить', 90),
 ('очень скучать', 87),
 ('хотеть домой', 87),
 ('быть счастливый', 87)

In [55]:
with open('w2v_all _clean.txt') as file:
    senti_words = [i.strip() for i in file.readlines()]

In [58]:
senti_words.extend([i.split('_')[0] for sublist in good_seed_tokens for i in sublist])

In [60]:
very_best_keywords = []
for i in cnt_multiword.most_common(1000):
    for word in senti_words:
        if word in i[0]:
            very_best_keywords.append(i[0])

In [ ]:
with open('result_rake_multiword.txt', 'w') as file:
    file.write(' '.join(set(very_best_keywords)))